<a href="https://colab.research.google.com/github/agmCorp/colab/blob/main/autodata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
import torch
from sentence_transformers import SentenceTransformer, util

# 1. INSTALACIÓN DE LIBRERÍAS
try:
    from sentence_transformers import SentenceTransformer
except:
    !pip install -U sentence-transformers
    from sentence_transformers import SentenceTransformer

# 2. CONFIGURACIÓN DE ARCHIVOS
FILE_BSE = '/content/sample_data/CART_MATRICERO_BSE_202602031858.csv'
FILE_AUTO = '/content/sample_data/CART_PAD_AUTODATA_202602031856.csv'

def cargar_datos():
    print("Cargando archivos CSV...")
    # Usamos low_memory=False por si hay columnas con tipos mixtos
    bse = pd.read_csv(FILE_BSE, encoding='utf-8')
    auto = pd.read_csv(FILE_AUTO, encoding='utf-8')
    return bse, auto

# 3. FUNCIONES DE APOYO
def limpiar_marca(t):
    # Remueve etiquetas tipo <BRA> y normaliza a mayúsculas
    return re.sub(r'<.*?>', '', str(t)).upper().strip()

def extraer_anios(rango_str):
    # Busca años de 4 dígitos en el string "2001 - 2003"
    anios = re.findall(r'\d{4}', str(rango_str))
    if len(anios) >= 2:
        return int(anios[0]), int(anios[-1])
    elif len(anios) == 1:
        return int(anios[0]), int(anios[0])
    return 0, 9999

# 4. EJECUCIÓN DEL PROCESO
try:
    df_bse, df_auto = cargar_datos()

    print("Normalizando datos...")
    # Limpieza de Marcas
    df_bse['MARCA_CLEAN'] = df_bse['MARCA'].apply(limpiar_marca)
    df_auto['MARCA_CLEAN'] = df_auto['MARCA_A'].apply(limpiar_marca)

    # Procesamiento de Rangos de Años en BSE
    df_bse[['ANIO_DESDE', 'ANIO_HASTA']] = df_bse['RANGO_ANIOS'].apply(
        lambda x: pd.Series(extraer_anios(x))
    )

    # Carga del Modelo Semántico
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Cargando modelo de IA en {device.upper()}...")
    model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device=device)

    resultados = []
    marcas_autodata = df_auto['MARCA_CLEAN'].unique()

    print("Iniciando matching inteligente (esto puede demorar unos minutos)...")

    for marca in marcas_autodata:
        # Filtrar por marca
        sub_bse = df_bse[df_bse['MARCA_CLEAN'] == marca].reset_index(drop=True)
        sub_auto = df_auto[df_auto['MARCA_CLEAN'] == marca]

        if sub_bse.empty:
            for _, row_a in sub_auto.iterrows():
                resultados.append({
                    'ID_AUTADATA': row_a['ID_AUTADATA'],
                    'ID_AUTODATA_BSE_SUG': None,
                    'DECISION': 'ALTA NECESARIA',
                    'SCORE': 0,
                    'MODELO_AUTODATA': row_a['MODELO_A'],
                    'MODELO_BSE_MATCH': 'MARCA NO ENCONTRADA'
                })
            continue

        # Generar vectores de BSE para esta marca
        bse_embeddings = model.encode(sub_bse['MODELO'].tolist(), convert_to_tensor=True)

        for _, row_a in sub_auto.iterrows():
            anio_val = row_a['ANIO_A']

            # Filtro temporal
            mask_anio = (sub_bse['ANIO_DESDE'] <= anio_val) & (sub_bse['ANIO_HASTA'] >= anio_val)
            indices_validos = np.where(mask_anio.values)[0]

            if len(indices_validos) == 0:
                resultados.append({
                    'ID_AUTADATA': row_a['ID_AUTADATA'],
                    'ID_AUTODATA_BSE_SUG': None,
                    'DECISION': 'ALTA NECESARIA',
                    'SCORE': 0,
                    'MODELO_AUTODATA': row_a['MODELO_A'],
                    'MODELO_BSE_MATCH': f'SIN RANGO PARA AÑO {anio_val}'
                })
                continue

            # Comparación semántica
            emb_a = model.encode(str(row_a['MODELO_A']), convert_to_tensor=True)
            hits = util.semantic_search(emb_a, bse_embeddings[indices_validos], top_k=1)

            best = hits[0][0]
            score = best['score']
            match_idx = indices_validos[best['corpus_id']]
            match_row = sub_bse.iloc[match_idx]

            # Definición de umbrales
            if score > 0.88:
                dec = 'YA EXISTE'
            elif score > 0.65:
                dec = 'REVISIÓN HUMANA'
            else:
                dec = 'ALTA NECESARIA'

            resultados.append({
                'ID_AUTADATA': row_a['ID_AUTADATA'],
                'ID_AUTODATA_BSE_SUG': match_row['ID_AUTODATA_BSE'],
                'DECISION': dec,
                'SCORE': round(score, 4),
                'MODELO_AUTODATA': row_a['MODELO_A'],
                'MODELO_BSE_MATCH': match_row['MODELO'],
                'ANIO_A': anio_val,
                'RANGO_BSE': match_row['RANGO_ANIOS']
            })

    # 5. GUARDAR RESULTADO
    df_final = pd.DataFrame(resultados)
    output_name = 'RESULTADO_MATCHING_IA.csv'
    df_final.to_csv(output_name, index=False, sep=';', encoding='utf-8-sig')

    print(f"\n✅ Proceso completado con éxito.")
    print(df_final['DECISION'].value_counts())
    print(f"\nDescarga el archivo: {output_name}")

except FileNotFoundError as e:
    print(f"❌ Error: No se encontró el archivo. Asegúrate de subir {FILE_BSE} y {FILE_AUTO} al panel izquierdo de Colab.")
except Exception as e:
    print(f"❌ Ocurrió un error inesperado: {e}")

Cargando archivos CSV...
Normalizando datos...
Cargando modelo de IA en CUDA...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Iniciando matching inteligente (esto puede demorar unos minutos)...

✅ Proceso completado con éxito.
DECISION
ALTA NECESARIA     10806
REVISIÓN HUMANA     8317
YA EXISTE            724
Name: count, dtype: int64

Descarga el archivo: RESULTADO_MATCHING_IA.csv


In [2]:
# Cargar el resultado que ya generamos
df_res = pd.read_csv('/content/RESULTADO_MATCHING_IA.csv', sep=';')

# 1. Aplicar umbral más agresivo (0.78 en lugar de 0.88)
df_res.loc[df_res['SCORE'] >= 0.78, 'DECISION'] = 'YA EXISTE'
df_res.loc[(df_res['SCORE'] < 0.78) & (df_res['SCORE'] >= 0.60), 'DECISION'] = 'REVISIÓN HUMANA'
df_res.loc[df_res['SCORE'] < 0.60, 'DECISION'] = 'ALTA NECESARIA'

print("--- NUEVOS NÚMEROS CON UMBRAL 0.78 ---")
print(df_res['DECISION'].value_counts())

# 2. Ver los casos más "dudosos" que quedaron para entender el patrón
print("\n--- MUESTRA DE REVISIÓN HUMANA (ZONA GRIS) ---")
display(df_res[df_res['DECISION'] == 'REVISIÓN HUMANA'][['MODELO_AUTODATA', 'MODELO_BSE_MATCH', 'SCORE']].head(15))

--- NUEVOS NÚMEROS CON UMBRAL 0.78 ---
DECISION
ALTA NECESARIA     8494
REVISIÓN HUMANA    7821
YA EXISTE          3532
Name: count, dtype: int64

--- MUESTRA DE REVISIÓN HUMANA (ZONA GRIS) ---


,MODELO_AUTODATA,MODELO_BSE_MATCH,SCORE
1,Damas Van Furg¿n c/diesel adaptado,VAN DAMAS (8 CC)FURGON,0.6148
2,Damas Van Furgon adaptada a rural,VAN DAMAS (ad Rural) (8 CC),0.7269
3,Espero CD 2.0 Full (DLX),ESPERO CD FULL,0.7641
4,Lanos S 1.5 dir 3p.,LANOS S 5P (15 CC),0.7388
5,Lanos S 1.5 3p.,LANOS S 5P (15 CC),0.7682
6,Lanos S 1.5 Full 4p.,LANOS S FULL-SEMI FULL 4P (15 CC),0.7766
9,Lanos SX 1.5 Full 4p.,LANOS SX FULL (15 CC),0.7180
10,Lanos SX 1.6 16v Full 3p.,LANOS SX FULL (15 CC),0.7525
11,Lanos SX 1.6 Full 3p.,LANOS SX FULL (15 CC),0.7444
12,Matiz SE,LANOS S(ADAPTADO) DIESEL,0.6579


In [3]:
import pandas as pd

# Cargar resultado
df = pd.read_csv('/content/RESULTADO_MATCHING_IA.csv', sep=';')

def reclasificar(row):
    score = row['SCORE']
    mod_a = str(row['MODELO_AUTODATA']).upper()
    mod_bse = str(row['MODELO_BSE_MATCH']).upper()

    # REGLA 1: Si el score es muy alto, ya está.
    if score >= 0.78:
        return 'YA EXISTE (Auto-aprobado)'

    # REGLA 2: Si uno está contenido en el otro (ignora ruido)
    # Ejemplo: "ESPERO CD FULL" está dentro de "ESPERO CD 2.0 FULL (DLX)"
    if mod_bse in mod_a or mod_a in mod_bse:
        if score > 0.55: # Solo si tienen un mínimo de relación
            return 'YA EXISTE (Match por Contenido)'

    # REGLA 3: Zona gris real
    if score >= 0.55:
        return 'REVISIÓN HUMANA'

    return 'ALTA NECESARIA'

# Aplicar la nueva lógica
df['NUEVA_DECISION'] = df.apply(reclasificar, axis=1)

print("--- COMPARATIVA DE RESULTADOS ---")
print("Antes:\n", df['DECISION'].value_counts())
print("\nAhora:\n", df['NUEVA_DECISION'].value_counts())

# Guardar para tu revisión final
df.to_csv('RESULTADO_OPTIMIZADO.csv', index=False, sep=';', encoding='utf-8-sig')

--- COMPARATIVA DE RESULTADOS ---
Antes:
 DECISION
ALTA NECESARIA     10806
REVISIÓN HUMANA     8317
YA EXISTE            724
Name: count, dtype: int64

Ahora:
 NUEVA_DECISION
REVISIÓN HUMANA                    9737
ALTA NECESARIA                     6459
YA EXISTE (Auto-aprobado)          3532
YA EXISTE (Match por Contenido)     119
Name: count, dtype: int64


In [5]:
import pandas as pd
import numpy as np
import re
import torch
from sentence_transformers import SentenceTransformer, util

# 1. INSTALACIÓN
try:
    from sentence_transformers import SentenceTransformer
except:
    !pip install -U sentence-transformers
    from sentence_transformers import SentenceTransformer

# 2. CONFIGURACIÓN DE ARCHIVOS
FILE_BSE = '/content/sample_data/CART_MATRICERO_BSE_202602031858.csv'
FILE_AUTO = '/content/sample_data/CART_PAD_AUTODATA_202602031856.csv'

# 3. FUNCIONES DE LIMPIEZA PROFUNDA
def limpiar_marca(t):
    return re.sub(r'<.*?>', '', str(t)).upper().strip()

def extraer_anios(rango_str):
    anios = re.findall(r'\d{4}', str(rango_str))
    if len(anios) >= 2: return int(anios[0]), int(anios[-1])
    elif len(anios) == 1: return int(anios[0]), int(anios[0])
    return 0, 9999

def limpiar_modelo_ruido(texto):
    """Limpia abreviaturas y caracteres rotos para mejorar el match"""
    t = str(texto).upper()
    # Reparar caracteres rotos comunes en exportaciones
    t = t.replace('¿', 'O').replace('Ñ', 'N')
    # Normalizar términos técnicos (ruido)
    t = re.sub(r'\b(C\.C\.|CC\.|C\.C)\b', 'CC', t)
    t = re.sub(r'\b(VALVULAS|V\.|16V|8V)\b', 'V', t)
    t = re.sub(r'\b(PUERTAS|P\.|PTAS|5P|4P|3P|2P)\b', 'P', t)
    t = re.sub(r'\b(AUTOMATICO|AUT\.|AUTO)\b', 'AUT', t)
    t = re.sub(r'\b(FULL|F\.FULL|EXTRA FULL)\b', 'FULL', t)
    t = re.sub(r'\b(PICK-UP|PICK UP|PKUP)\b', 'PU', t)
    # Quitar puntos, comas y espacios extra
    t = re.sub(r'[.,;()]', ' ', t)
    return " ".join(t.split())

# 4. PROCESO PRINCIPAL
print("Cargando y pre-procesando datos...")
df_bse = pd.read_csv(FILE_BSE)
df_auto = pd.read_csv(FILE_AUTO)

df_bse['MARCA_CLEAN'] = df_bse['MARCA'].apply(limpiar_marca)
df_auto['MARCA_CLEAN'] = df_auto['MARCA_A'].apply(limpiar_marca)
df_bse[['ANIO_DESDE', 'ANIO_HASTA']] = df_bse['RANGO_ANIOS'].apply(lambda x: pd.Series(extraer_anios(x)))

# Aplicamos la limpieza de ruido a las descripciones para la comparación
df_bse['MOD_ST'] = df_bse['MODELO'].apply(limpiar_modelo_ruido)
df_auto['MOD_ST'] = df_auto['MODELO_A'].apply(limpiar_modelo_ruido)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device=device)

resultados = []
marcas = df_auto['MARCA_CLEAN'].unique()

print(f"Iniciando Matching Inteligente en {device}...")

for marca in marcas:
    sub_bse = df_bse[df_bse['MARCA_CLEAN'] == marca].reset_index(drop=True)
    sub_auto = df_auto[df_auto['MARCA_CLEAN'] == marca]

    if sub_bse.empty:
        for _, row_a in sub_auto.iterrows():
            resultados.append({'ID_AUTADATA': row_a['ID_AUTADATA'], 'DECISION': 'ALTA NECESARIA', 'SCORE': 0, 'MODELO_AUTODATA': row_a['MODELO_A'], 'MODELO_BSE_MATCH': 'MARCA NO EXISTE'})
        continue

    # Encode de modelos BSE (usando la versión limpia de ruido)
    bse_embs = model.encode(sub_bse['MOD_ST'].tolist(), convert_to_tensor=True)

    for _, row_a in sub_auto.iterrows():
        anio = row_a['ANIO_A']
        mask = (sub_bse['ANIO_DESDE'] <= anio) & (sub_bse['ANIO_HASTA'] >= anio)
        valid_idx = np.where(mask.values)[0]

        if len(valid_idx) == 0:
            resultados.append({'ID_AUTADATA': row_a['ID_AUTADATA'], 'DECISION': 'ALTA NECESARIA', 'SCORE': 0, 'MODELO_AUTODATA': row_a['MODELO_A'], 'MODELO_BSE_MATCH': f'AÑO {anio} FUERA DE RANGO'})
            continue

        # Encode de modelo Autodata (limpio)
        emb_a = model.encode(row_a['MOD_ST'], convert_to_tensor=True)
        hits = util.semantic_search(emb_a, bse_embs[valid_idx], top_k=1)

        best = hits[0][0]
        score = best['score']
        match_row = sub_bse.iloc[valid_idx[best['corpus_id']]]

        # --- LÓGICA DE DECISIÓN DEFINITIVA ---
        mod_a_st = row_a['MOD_ST']
        mod_b_st = match_row['MOD_ST']

        # REGLA 1: Match Exacto tras limpieza de ruido o Inclusión Directa
        if (mod_a_st == mod_b_st) or (mod_b_st in mod_a_st) or (mod_a_st in mod_b_st):
            if score > 0.60:
                decision = 'YA EXISTE (Match Técnico)'
                score = max(score, 0.95) # Forzamos score alto por coincidencia de texto
            else:
                decision = 'REVISION HUMANA'
        # REGLA 2: Score Semántico Puro
        elif score >= 0.78:
            decision = 'YA EXISTE (Alta Confianza)'
        elif score >= 0.55:
            decision = 'REVISION HUMANA'
        else:
            decision = 'ALTA NECESARIA'

        resultados.append({
            'ID_AUTADATA': row_a['ID_AUTADATA'],
            'ID_AUTODATA_BSE_SUG': match_row['ID_AUTODATA_BSE'],
            'DECISION': decision,
            'SCORE': round(score, 4),
            'MODELO_AUTODATA': row_a['MODELO_A'],
            'MODELO_BSE_MATCH': match_row['MODELO'],
            'ANIO': anio,
            'RANGO_BSE': match_row['RANGO_ANIOS']
        })

# 5. SALIDA
df_fin = pd.DataFrame(resultados)
df_fin.to_csv('MATCHING_FINAL_REVISADO.csv', index=False, sep=';', encoding='utf-8-sig')
print("\n--- RESUMEN FINAL ---")
print(df_fin['DECISION'].value_counts())
print("\nArchivo 'MATCHING_FINAL_REVISADO.csv' listo para descargar.")

Cargando y pre-procesando datos...


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Iniciando Matching Inteligente en cuda...

--- RESUMEN FINAL ---
DECISION
YA EXISTE (Alta Confianza)    8897
REVISION HUMANA               7090
ALTA NECESARIA                3442
YA EXISTE (Match Técnico)      418
Name: count, dtype: int64

Archivo 'MATCHING_FINAL_REVISADO.csv' listo para descargar.


In [1]:
# =========================
# COLAB - MATCHING BSE vs AUTODATA (conservador, "mejor no estoy seguro")
# Copiar/pegar tal cual.
# =========================

import re
import numpy as np
import pandas as pd

# ---------- (1) Instalar deps ----------
try:
    import torch
    from sentence_transformers import SentenceTransformer, util
except Exception:
    !pip -q install -U sentence-transformers
    import torch
    from sentence_transformers import SentenceTransformer, util

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# ---------- (2) RUTAS (ajustá si tus archivos están en otra carpeta) ----------
FILE_BSE  = "/content/sample_data/CART_MATRICERO_BSE_202602031858.csv"
FILE_AUTO = "/content/sample_data/CART_PAD_AUTODATA_202602031856.csv"

OUT_FILE  = "MATCHING_FINAL_CONSERVADOR.csv"


# ---------- (3) Limpieza base ----------
def limpiar_marca(t: str) -> str:
    # Remueve tags tipo <BRA>, etc.
    return re.sub(r"<.*?>", "", str(t)).upper().strip()

def extraer_anios(rango_str: str):
    anios = re.findall(r"\d{4}", str(rango_str))
    if len(anios) >= 2:
        return int(anios[0]), int(anios[-1])
    if len(anios) == 1:
        y = int(anios[0])
        return y, y
    return 0, 9999

def normalize_text(t: str) -> str:
    # Normalización práctica para catálogos
    s = str(t).upper()

    # caracteres raros comunes
    s = s.replace("¿", "O").replace("Ñ", "N")

    # separadores a espacio
    s = re.sub(r"[.,;()/_\-]", " ", s)

    # normalizar tokens frecuentes
    s = re.sub(r"\b(AUTOMATICO|AUTOMÁTICO|AUT\.|AUTO)\b", "AUT", s)
    s = re.sub(r"\b(MANUAL|MECANICA|MECÁNICA|MT)\b", "MAN", s)

    # puertas
    s = re.sub(r"\b(\d)\s*(PUERTAS|PUERTA|P\.|PTAS)\b", r"\1P", s)
    s = re.sub(r"\b(\d)P(UERTAS)?\b", r"\1P", s)

    # turbo
    s = re.sub(r"\b(TURBO|T\.|T)\b", "TURBO", s)

    # full
    s = re.sub(r"\b(EXTRA\s*FULL|FULL)\b", "FULL", s)

    # limpiar espacios
    s = re.sub(r"\s+", " ", s).strip()
    return s


# ---------- (4) Extraer features "duras" del texto ----------
def parse_doors(s: str):
    # devuelve int o None
    m = re.search(r"\b([2-5])P\b", s)
    return int(m.group(1)) if m else None

def parse_trans(s: str):
    # AUT / MAN / None
    if re.search(r"\bAUT\b", s):
        return "AUT"
    if re.search(r"\bMAN\b", s):
        return "MAN"
    return None

def parse_engine_liters(s: str):
    # detecta 1.4 / 1,4 / 14 (según como venga)
    # Preferimos patrones con separador.
    m = re.search(r"\b(\d)\s*[\.,]\s*(\d)\b", s)
    if m:
        return float(f"{m.group(1)}.{m.group(2)}")
    # A veces viene 14 -> 1.4, 20 -> 2.0 (riesgoso). Lo dejo apagado por default.
    return None

def parse_turbo(s: str):
    return True if re.search(r"\bTURBO\b", s) else False

def feature_score(a_feat, b_feat):
    """
    score en [0,1] según coincidencia de features.
    Penaliza si hay contradicción explícita.
    """
    score = 0.0
    weight = 0.0

    # doors
    if a_feat["doors"] is not None and b_feat["doors"] is not None:
        weight += 1.0
        score += 1.0 if a_feat["doors"] == b_feat["doors"] else 0.0

    # trans
    if a_feat["trans"] is not None and b_feat["trans"] is not None:
        weight += 1.0
        score += 1.0 if a_feat["trans"] == b_feat["trans"] else 0.0

    # engine liters (si está en ambos)
    if a_feat["engine"] is not None and b_feat["engine"] is not None:
        weight += 1.0
        score += 1.0 if abs(a_feat["engine"] - b_feat["engine"]) < 0.05 else 0.0

    # turbo
    # Solo puntúa si ambos tienen señal fuerte.
    if a_feat["turbo"] is True and b_feat["turbo"] is True:
        weight += 0.5
        score += 0.5
    elif a_feat["turbo"] is False and b_feat["turbo"] is False:
        # no sumo: "no turbo" muchas veces no se explicita; evitar sesgo
        pass
    elif (a_feat["turbo"] is True and b_feat["turbo"] is False) or (a_feat["turbo"] is False and b_feat["turbo"] is True):
        # contradicción explícita: penaliza si ambos tienen turbo explícito (acá solo vemos TURBO token)
        # si uno no lo tiene, podría ser omisión, entonces no penalizo duro.
        pass

    if weight == 0.0:
        return None  # sin evidencia
    return score / weight


def extract_features(norm_text: str):
    return {
        "doors": parse_doors(norm_text),
        "trans": parse_trans(norm_text),
        "engine": parse_engine_liters(norm_text),
        "turbo": parse_turbo(norm_text),
    }


# ---------- (5) Similaridad textual (char ngrams) ----------
# Lo armamos por marca (y por pool BSE ya filtrado por año).
def char_ngram_similarity(query_text: str, candidates_texts: list, vectorizer: TfidfVectorizer):
    # devuelve similitudes coseno contra cada candidato (np.array)
    X = vectorizer.transform(candidates_texts)
    q = vectorizer.transform([query_text])
    sims = cosine_similarity(q, X).ravel()
    return sims


# ---------- (6) Decisión conservadora ----------
def decide_conservative(score_final, score1, score2, feat_ok, text_ok, margin=0.03):
    """
    Prioridad: evitar falsos positivos.
    - Si hay duda: REVISION HUMANA.
    """
    # Reglas duras primero
    if not feat_ok:
        # contradicción dura -> nunca "ya existe"
        return "REVISION HUMANA"

    # umbrales conservadores
    if score_final >= 0.86 and score1 >= 0.82 and text_ok and (score1 - score2) >= margin:
        return "YA EXISTE"
    if score_final >= 0.72:
        return "REVISION HUMANA"
    return "ALTA NECESARIA"


# ---------- (7) Carga ----------
print("Cargando CSVs...")
df_bse  = pd.read_csv(FILE_BSE)
df_auto = pd.read_csv(FILE_AUTO)

# Normalizaciones
df_bse["MARCA_CLEAN"]  = df_bse["MARCA"].apply(limpiar_marca)
df_auto["MARCA_CLEAN"] = df_auto["MARCA_A"].apply(limpiar_marca)

df_bse[["ANIO_DESDE","ANIO_HASTA"]] = df_bse["RANGO_ANIOS"].apply(lambda x: pd.Series(extraer_anios(x)))

df_bse["TXT_NORM"]  = df_bse["MODELO"].apply(normalize_text)
df_auto["TXT_NORM"] = df_auto["MODELO_A"].apply(normalize_text)

df_bse["FEAT"]  = df_bse["TXT_NORM"].apply(extract_features)
df_auto["FEAT"] = df_auto["TXT_NORM"].apply(extract_features)

# Asegurar ANIO_A como int (si viene como str)
df_auto["ANIO_A"] = df_auto["ANIO_A"].astype(int)

print("Listo.\n")


# ---------- (8) Modelo embeddings ----------
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Cargando modelo embeddings en:", device)
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device=device)


# ---------- (9) Proceso por marca ----------
TOP_K = 5

resultados = []
marcas = sorted(df_auto["MARCA_CLEAN"].unique().tolist())

print(f"Marcas a procesar: {len(marcas)}")

for marca in marcas:
    sub_bse = df_bse[df_bse["MARCA_CLEAN"] == marca].reset_index(drop=True)
    sub_auto = df_auto[df_auto["MARCA_CLEAN"] == marca].reset_index(drop=True)

    if sub_bse.empty:
        # Marca no existe en BSE => todo alta necesaria
        for _, row_a in sub_auto.iterrows():
            resultados.append({
                "ID_AUTADATA": row_a["ID_AUTADATA"],
                "DECISION": "ALTA NECESARIA",
                "SCORE_FINAL": 0.0,
                "SCORE_EMB_1": 0.0,
                "SCORE_EMB_2": 0.0,
                "SCORE_TEXT_1": 0.0,
                "SCORE_FEAT_1": None,
                "ID_AUTODATA_BSE_1": None,
                "ID_AUTODATA_BSE_2": None,
                "ID_AUTODATA_BSE_3": None,
                "MODELO_AUTODATA": row_a["MODELO_A"],
                "OBS": "MARCA NO EXISTE EN BSE"
            })
        continue

    # Pre-encode embeddings del pool BSE de la marca
    bse_texts = sub_bse["TXT_NORM"].tolist()
    bse_embs = model.encode(bse_texts, convert_to_tensor=True)

    # Vectorizer char ngrams para la marca (se ajusta al vocabulario de esa marca)
    vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(3,5), min_df=1)
    vectorizer.fit(bse_texts)

    for _, row_a in sub_auto.iterrows():
        anio = int(row_a["ANIO_A"])

        # Filtro por rango de años
        mask = (sub_bse["ANIO_DESDE"] <= anio) & (sub_bse["ANIO_HASTA"] >= anio)
        valid_idx = np.where(mask.values)[0]

        if len(valid_idx) == 0:
            resultados.append({
                "ID_AUTADATA": row_a["ID_AUTADATA"],
                "DECISION": "ALTA NECESARIA",
                "SCORE_FINAL": 0.0,
                "SCORE_EMB_1": 0.0,
                "SCORE_EMB_2": 0.0,
                "SCORE_TEXT_1": 0.0,
                "SCORE_FEAT_1": None,
                "ID_AUTODATA_BSE_1": None,
                "ID_AUTODATA_BSE_2": None,
                "ID_AUTODATA_BSE_3": None,
                "MODELO_AUTODATA": row_a["MODELO_A"],
                "OBS": f"AÑO {anio} FUERA DE RANGO EN BSE"
            })
            continue

        # Embedding query
        emb_a = model.encode(row_a["TXT_NORM"], convert_to_tensor=True)

        # Top-k por embedding dentro del pool válido por año
        hits = util.semantic_search(emb_a, bse_embs[valid_idx], top_k=min(TOP_K, len(valid_idx)))[0]
        # hits: [{'corpus_id': idx_in_valid, 'score': ...}, ...]

        # Armar lista de candidatos reales en sub_bse
        cand_rows = []
        for h in hits:
            ridx = valid_idx[h["corpus_id"]]  # índice real en sub_bse
            cand_rows.append((ridx, float(h["score"])))

        # Si por algo no hay hits
        if not cand_rows:
            resultados.append({
                "ID_AUTADATA": row_a["ID_AUTADATA"],
                "DECISION": "ALTA NECESARIA",
                "SCORE_FINAL": 0.0,
                "SCORE_EMB_1": 0.0,
                "SCORE_EMB_2": 0.0,
                "SCORE_TEXT_1": 0.0,
                "SCORE_FEAT_1": None,
                "ID_AUTODATA_BSE_1": None,
                "ID_AUTODATA_BSE_2": None,
                "ID_AUTODATA_BSE_3": None,
                "MODELO_AUTODATA": row_a["MODELO_A"],
                "OBS": "SIN CANDIDATOS"
            })
            continue

        # Text sim char-ngrams SOLO sobre top-k para no gastar
        cand_texts = [sub_bse.loc[i, "TXT_NORM"] for (i, _) in cand_rows]
        text_sims = char_ngram_similarity(row_a["TXT_NORM"], cand_texts, vectorizer)

        # Feature sims
        a_feat = row_a["FEAT"]
        feat_sims = []
        for (i, _) in cand_rows:
            b_feat = sub_bse.loc[i, "FEAT"]
            fs = feature_score(a_feat, b_feat)
            feat_sims.append(fs)

        # Score final combinado (explicable):
        # - embedding 0.5
        # - char ngrams 0.35
        # - features 0.15 (si hay evidencia)
        # Conservador: si features contradicen (cuando hay evidencia), no dejamos "YA EXISTE".
        final_scores = []
        for k, (i, emb_sc) in enumerate(cand_rows):
            txt_sc = float(text_sims[k])
            fs = feat_sims[k]

            if fs is None:
                # sin evidencia de features -> no fuerzo
                score_final = 0.55*emb_sc + 0.45*txt_sc
            else:
                score_final = 0.50*emb_sc + 0.35*txt_sc + 0.15*fs

            final_scores.append(score_final)

        # Elegir candidato por SCORE_FINAL (no por embedding)
        best_k = int(np.argmax(final_scores))
        best_idx, best_emb = cand_rows[best_k]
        best_txt = float(text_sims[best_k])
        best_fs  = feat_sims[best_k]
        best_final = float(final_scores[best_k])

        # segundo mejor para margen de ambigüedad
        sorted_final = sorted(final_scores, reverse=True)
        score2_final = float(sorted_final[1]) if len(sorted_final) > 1 else 0.0

        # checks conservadores
        # feat_ok: si hay features en ambos y NO coinciden => no ok
        feat_ok = True
        if best_fs is not None and best_fs < 0.50:
            feat_ok = False  # contradicción fuerte en al menos 1 feature duro

        # text_ok: exige un mínimo textual para decir "ya existe"
        text_ok = (best_txt >= 0.55)  # conservador, ajustable

        decision = decide_conservative(
            score_final=best_final,
            score1=best_final,
            score2=score2_final,
            feat_ok=feat_ok,
            text_ok=text_ok,
            margin=0.03
        )

        # Guardar top-3 candidatos para revisión humana
        # ordenados por final score
        ranked = sorted(
            [(cand_rows[k][0], cand_rows[k][1], float(text_sims[k]), feat_sims[k], float(final_scores[k])) for k in range(len(cand_rows))],
            key=lambda x: x[4],
            reverse=True
        )

        def safe_get(rank, field):
            if rank < len(ranked):
                return ranked[rank][field]
            return None

        # Campos del top1/top2/top3
        id1 = sub_bse.loc[safe_get(0,0), "ID_AUTODATA_BSE"] if safe_get(0,0) is not None else None
        id2 = sub_bse.loc[safe_get(1,0), "ID_AUTODATA_BSE"] if safe_get(1,0) is not None else None
        id3 = sub_bse.loc[safe_get(2,0), "ID_AUTODATA_BSE"] if safe_get(2,0) is not None else None

        # Observación útil
        obs = ""
        if decision == "YA EXISTE":
            obs = "OK (conservador)"
        elif decision == "REVISION HUMANA":
            # decir por qué
            if (best_final >= 0.72) and ((best_final - score2_final) < 0.03):
                obs = "AMBIGUO (scores cercanos)"
            elif not feat_ok:
                obs = "CONTRADICCION FEATURES"
            else:
                obs = "ZONA GRIS"
        else:
            obs = "BAJA SIMILITUD"

        resultados.append({
            "ID_AUTADATA": row_a["ID_AUTADATA"],
            "ANIO_A": anio,
            "MARCA": marca,
            "MODELO_AUTODATA": row_a["MODELO_A"],

            "DECISION": decision,
            "SCORE_FINAL": round(best_final, 4),

            # TOP1
            "ID_AUTODATA_BSE_1": id1,
            "SCORE_EMB_1": round(float(safe_get(0,1)) if safe_get(0,1) is not None else 0.0, 4),
            "SCORE_TEXT_1": round(float(safe_get(0,2)) if safe_get(0,2) is not None else 0.0, 4),
            "SCORE_FEAT_1": (round(float(safe_get(0,3)), 4) if safe_get(0,3) is not None else None),
            "MODELO_BSE_1": (sub_bse.loc[safe_get(0,0), "MODELO"] if safe_get(0,0) is not None else None),
            "RANGO_BSE_1": (sub_bse.loc[safe_get(0,0), "RANGO_ANIOS"] if safe_get(0,0) is not None else None),

            # TOP2
            "ID_AUTODATA_BSE_2": id2,
            "SCORE_FINAL_2": (round(float(safe_get(1,4)), 4) if safe_get(1,4) is not None else None),
            "MODELO_BSE_2": (sub_bse.loc[safe_get(1,0), "MODELO"] if safe_get(1,0) is not None else None),

            # TOP3
            "ID_AUTODATA_BSE_3": id3,
            "SCORE_FINAL_3": (round(float(safe_get(2,4)), 4) if safe_get(2,4) is not None else None),
            "MODELO_BSE_3": (sub_bse.loc[safe_get(2,0), "MODELO"] if safe_get(2,0) is not None else None),

            "OBS": obs
        })

print("\nArmando salida...")
df_out = pd.DataFrame(resultados)

# Orden sugerido: primero los dudosos arriba
order_map = {"REVISION HUMANA": 0, "ALTA NECESARIA": 1, "YA EXISTE": 2}
df_out["__ord"] = df_out["DECISION"].map(order_map).fillna(9).astype(int)
df_out = df_out.sort_values(["__ord", "MARCA", "ANIO_A"], ascending=[True, True, True]).drop(columns="__ord")

df_out.to_csv(OUT_FILE, index=False, sep=";", encoding="utf-8-sig")

print("Listo:", OUT_FILE)
print("\nResumen decisiones:")
print(df_out["DECISION"].value_counts())


Cargando CSVs...
Listo.

Cargando modelo embeddings en: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/526 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Marcas a procesar: 198

Armando salida...
Listo: MATCHING_FINAL_CONSERVADOR.csv

Resumen decisiones:
DECISION
ALTA NECESARIA     16315
REVISION HUMANA     2796
YA EXISTE            736
Name: count, dtype: int64
